Will need to pip install
* landsatxplore
* ipypb

In [ ]:
!pip install landsatxplore
!pip install ipypb

In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import pickle
from ipypb import track
from ipypb import chain
import glob

### Set directories

In [ ]:
os.chdir("C:/Users/User/University College London/O'Sullivan, Aidan - SDG6/")
s_path = './Landsat data/Scenes/'

### Read in site data with coordinates

In [2]:
sites = pd.read_pickle("sites.p")
sites.head(2)

wb_cat determinand_id  value       lon       lat  \
sid          dt                                                            
ATSE20101000 2009-05-12     LW  EEA_3164-01-0    5.0  14.12236  46.63092   
             2009-08-19     LW  EEA_3164-01-0    2.0  14.12236  46.63092   

                                          geometry checked  \
sid          dt                                              
ATSE20101000 2009-05-12  POINT (14.12236 46.63092)    True   
             2009-08-19  POINT (14.12236 46.63092)    True   

                                      scene_id aquisition_dt  \
sid          dt                                                
ATSE20101000 2009-05-12                    NaN           NaN   
             2009-08-19  LT51910272009229MOR00           NaN   

                                    scene_dt  
sid          dt                               
ATSE20101000 2009-05-12                  NaN  
             2009-08-19  2009-08-17T00:00:00

In [10]:
### Change checked back to False for site/date with no retrieved scenes

sites['checked'] = pd.Series(np.bool)
sites['checked'] = np.where(sites.scene_id.isna(),False,True)

wb_cat determinand_id  value       lon       lat  \
sid          dt                                                            
ATSE20101000 2009-05-12     LW  EEA_3164-01-0    5.0  14.12236  46.63092   
             2009-08-19     LW  EEA_3164-01-0    2.0  14.12236  46.63092   
             2009-12-16     LW  EEA_3164-01-0   10.0  14.12236  46.63092   
             2010-03-18     LW  EEA_3164-01-0    6.0  14.12236  46.63092   
             2010-05-26     LW  EEA_3164-01-0    2.0  14.12236  46.63092   

                                          geometry  checked  \
sid          dt                                               
ATSE20101000 2009-05-12  POINT (14.12236 46.63092)    False   
             2009-08-19  POINT (14.12236 46.63092)    False   
             2009-12-16  POINT (14.12236 46.63092)    False   
             2010-03-18  POINT (14.12236 46.63092)    False   
             2010-05-26  POINT (14.12236 46.63092)     True   

                                      scene_id        aquisition_dt  
sid          dt                                                      
ATSE20101000 2009-05-12                    NaN                  NaN  
             2009-08-19                    NaN                  NaN  
             2009-12-16                    NaN                  NaN  
             2010-03-18                    NaN                  NaN  
             2010-05-26  LE71910282010144ASN00  2010-05-24T09:43:12

### Landsat API call to check for coverage

https://earthexplorer.usgs.gov/inventory/documentation#access

https://pypi.org/project/landsatxplore/

In [13]:
import landsatxplore.api
from landsatxplore.earthexplorer import EarthExplorer

user = 'clement.attwood.18@ucl.ac.uk'
password = 'm9.iETxKMtZ6B,q'

api = landsatxplore.api.API(user,password)

In [ ]:
# Create call for each site and each date

lag = 8                      # number of days to search either side of sample date
max_cloud_cover = 20         # max cloud cover for scene

platforms = ['LANDSAT_ETM_C1','LANDSAT_TM_C1','LANDSAT_8_C1'] 

for sid,dt in track(sites[sites.checked==False].index[5001:15174]):
     
    ## set checked to 1
    date = dt.strftime("%Y-%m-%d")
    sites.loc[(sid,date),'checked'] = 1
    
    ## set start end dates of search
    start = dt-datetime.timedelta(days=lag)
    end = dt+datetime.timedelta(days=lag)
    
    sites.loc[sid,'checked'] = True
    
    for platform in platforms:

        scenes = api.search(dataset = platform,
                            latitude = sites.loc[sid,'lat'][0],
                            longitude = sites.loc[sid,'lon'][0],
                            start_date = start.strftime("%Y-%m-%d"),
                            end_date = end.strftime("%Y-%m-%d"),
                            max_cloud_cover = max_cloud_cover)
                 
        if len(scenes)>0:
            sites.loc[(sid,date),'scene_id'] = scenes[0]['entityId']
            sites.loc[(sid,date),'scene_dt'] = scenes[0]['modifiedDate']
            pickle.dump(scenes,open(s_path+'scene_'+str(scenes[0]['entityId'])+'.p','wb'))         

pickle.dump(sites,open('sites.p','wb'))

 [█###########################################################] 279/13491 [05:33:49<01:12, 71.79s/it]

In [16]:
pickle.dump(sites,open('sites.p','wb'))